The code below selects 16 128x128 tiles for each image and mask based on the maximum number of tissue pixels. The kernel also provides computed image stats. Please check my kernels to see how to use this data. 
![](https://i.ibb.co/RzSWP56/convert.png)

# Imports

In [ ]:
import os
import cv2
import skimage.io
from tqdm.notebook import tqdm
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from glob import glob
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
import tensorflow as tf



## Setting the seeds for Reproducibility.
seed = 3141
np.random.seed(seed)

# Loading Data

In [ ]:
df = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv')
df = df[df['data_provider'] == 'karolinska'].reset_index(drop=True)
ids = df['image_id'].values
df.head(60)

# Constant Varibles

In [ ]:
TRAIN = '../input/prostate-cancer-grade-assessment/train_images/'
MASKS = '../input/prostate-cancer-grade-assessment/train_label_masks/'
OUT_TRAIN = 'train.zip'
OUT_MASKS = 'masks.zip'

# Modifable Variables

In [ ]:
sz = 120
N = 20
patients = 1000

# Demo of Size of Images

In [ ]:
img = skimage.io.MultiImage(os.path.join(TRAIN,ids[0]+'.tiff'))
img[0].shape, img[1].shape, img[2].shape

# Function to create tiles

In [ ]:
def tile(img, mask):
    result = []
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=255)
    mask = np.pad(mask,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                constant_values=0)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    mask = mask.reshape(mask.shape[0]//sz,sz,mask.shape[1]//sz,sz,3)
    mask = mask.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        mask = np.pad(mask,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=0)
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    mask = mask[idxs]
    return img, mask


# create_tiles and Labels

In [ ]:
X = []
Y = []
for name in tqdm(ids[0:patients], total=patients):
    img = skimage.io.MultiImage(os.path.join(TRAIN,name+'.tiff'))[-1]
    mask = skimage.io.MultiImage(os.path.join(MASKS,name+'_mask.tiff'))[-1]
    img, mask = tile(img,mask)
    k = 0
    while k < N:
        label = 0
        if 2 in np.unique(mask[k, ]):
            label = 1
        X.append(img[k, ])
        Y.append(label)
        k += 1

In [ ]:
X = np.array(X)
Y = np.array(Y)

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X, Y,random_state=0,test_size=0.25)

# Fill Remaining Notebook.....!

In [ ]:
print('X shape:', X.shape)
print('Y_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

In [ ]:
plt.figure(figsize=(20,8))
for i in range(10,18):
    plt.subplot(231 + (i))
    plt.imshow(X_train[i])

In [ ]:
from keras.utils import Sequence
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose)

In [ ]:
image_dim=(120,120,3)
BATCH_SIZE=50

In [ ]:

class Generator(Sequence):
    def __init__(self,input_data,batch_size=BATCH_SIZE,dims=image_dim,is_shuffle=True,n_classes=3,is_train=True):
        self.image_ids=input_data[0]
        self.labels=input_data[1]
        self.batch_size=batch_size
        self.dims=image_dim
        self.shuffle=is_shuffle
        self.n_classes=n_classes
        self.is_train=is_train
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.image_ids) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        image_ids_temp = [self.image_ids[k] for k in indexes]
        labels_temp = [self.labels[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(image_ids_temp,labels_temp)

        return X, y
    
    def augment_flips_color(self,p=.5):
        return Compose([
            Flip(),
            RandomRotate90(),
            Transpose(),
            HorizontalFlip(),
            ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
            Blur(blur_limit=3),
        ], p=p)
    
    def __data_generation(self, list_IDs_temp,lbls):
        X = np.zeros((self.batch_size, *self.dims))
        y = np.zeros((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img=MultiImage(os.path.join(main_path,'train_images',ID+'.tiff'))
            img = resize(img[-1], (self.dims[0], self.dims[1]))
            #Augmentation
            if self.is_train:
                aug = self.augment_flips_color(p=1)
                img = aug(image=img)['image']
                
            X[i] = img

            # Store class
            y[i] = lbls[i]

        return X, to_categorical(y, num_classes=self.n_classes)

In [ ]:
#convert values to float as result will be a float. If not done vals are set to zero
X_train = X_train.astype("float32")/255
X_test = X_test.astype("float32")/255




In [ ]:
#notice num_classes is set to 2 as we have 2 different labels
Y_train = to_categorical(Y_train, num_classes=2)
Y_test = to_categorical(Y_test, num_classes=2)

In [ ]:

from keras.layers.convolutional import SeparableConv2D


In [ ]:
channelDim = -1

model=Sequential()
model.add(SeparableConv2D(30, (3,3), padding="same",input_shape=(120,120,3)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(SeparableConv2D(60, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(60, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(SeparableConv2D(120, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(120, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(SeparableConv2D(240, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=channelDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))


model.add(Flatten())
model.add(Dense(240))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(1))
model.add(Activation("sigmoid")) 
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)


In [ ]:
dot_img_file = '/tmp/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:

model.compile(loss='binary_crossentropy',optimizer=Adam (0.001),metrics=['acc'])

In [ ]:
history=model.fit(X,Y,epochs=5,validation_split=0.25,batch_size=50)

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train_data','test_data'])
plt.title('loss analysis')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['train_data','test_data'])
plt.title('accuracy analysis')
plt.show()

In [ ]:
img = X_test[0, ]
print (img.shape)

In [ ]:
plt.figure(figsize=(20,8))
plt.subplot(241)
plt.imshow(img[:, :, 0], cmap="gray")